In [1]:
import os
import os
import random
import torch
import cv2
import numpy as np
from lungmask import mask
import SimpleITK as sitk
import torchvision.transforms as transforms
from scipy.ndimage.filters import median_filter
import pydicom
from torch.utils.data import Dataset
from itertools import combinations
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch import optim
import torch.nn.functional as F

C:\Users\Nimesha\AppData\Local\Temp\ipykernel_14288\2416076515.py:10: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import median_filter


In [2]:


def transform_to_hu(medical_image, image):
    intercept = medical_image.RescaleIntercept
    slope = medical_image.RescaleSlope
    hu_image = image * slope + intercept
    return hu_image

def get_mask(filename, plot_mask=False, return_val=False):

    input_image = sitk.ReadImage(filename)
    mask_out = mask.apply(input_image)[0]  #default model is U-net(R231)

    if return_val:
        return mask_out

def preprocess_images(img,dicom_image):

    hu_image = transform_to_hu(dicom_image, img)
    filtered_image = median_filter(hu_image, size=(3, 3))
    return filtered_image

def transform(dcm_path):

    dicom_image= pydicom.dcmread(dcm_path)

    image = np.array(dicom_image.pixel_array)
    print(image.shape)

    cleaned_image = preprocess_images(image,dicom_image)
    masked_img=get_mask(dcm_path,plot_mask=True,return_val=True)
    
    mask_on_orginal = cleaned_image * masked_img
    mask_on_orginal = cv2.resize(mask_on_orginal, (224, 224))
       
    image = mask_on_orginal.astype('float32')
    image = np.expand_dims(image, axis=0)
       
    image = torch.from_numpy(image)

    return image

In [3]:
#create the Siamese Neural Network
class SiameseNetwork(nn.Module):

    def __init__(self):
        
        super(SiameseNetwork, self).__init__()

        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
          
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(64, 1024, kernel_size=3,stride=1),
            nn.ReLU(inplace=True)
        )

        # Calculate output shape of the convolutional layers
        conv_output_shape = self.cnn1(torch.zeros(1, *(1, 224, 224))).shape

        # Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(
            nn.Linear(conv_output_shape[1] * conv_output_shape[2] * conv_output_shape[3], 1024),
            nn.ReLU(inplace=True),
            
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            
            nn.Linear(256,2)
        )
        
    def forward_once(self, x):
        
       
        #  This function will be called for both images
        # Its output is used to determine the similiarity
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

       

    def forward(self, input1):
        # In this function we pass in both images and obtain both vectors
        # which are returned
        output1 = self.forward_once(input1)
        
        
        return output1

In [4]:
folder_path = 'C:/Users/Nimesha/Documents/MSC_RESEARCH/IMAGES/SNN_IMAGES'
image_torch = []
labels = []

image_files = os.listdir(folder_path)

for image_file in image_files:
     
    label_ch = image_file[0]
    
    if label_ch =='A' :
            label=1
    elif label_ch =='B':
            label=2
    elif label_ch =='G':  
            label=3 
    elif label_ch =='E':
            label=4  
    else : label=5



    dicom_file= os.path.join(folder_path, image_file)
    dicom_image=pydicom.dcmread(dicom_file)
    image = np.array(dicom_image.pixel_array)
    

    cleaned_image = preprocess_images(image,dicom_image)
    masked_img=get_mask(dicom_file,plot_mask=True,return_val=True)
    
    mask_on_orginal = cleaned_image * masked_img
    mask_on_orginal = cv2.resize(mask_on_orginal, (224, 224))
       
    image = mask_on_orginal.astype('float32')
    image = np.expand_dims(image, axis=0)
       
    image = torch.from_numpy(image)

    image_torch.append(image)
    labels.append(label)

  
    




100%|██████████| 2/2 [00:00<?, ?it/s]


In [5]:
# Convert the image tensors and labels into a single batch tensor
images = torch.stack( image_torch)
labels = torch.tensor(labels)

In [6]:
# Forward pass to extract features

net = SiameseNetwork().cuda()
output_features=net(images)


from sklearn.decomposition import PCA

pca = PCA(n_components=2)
reduced_features = pca.fit_transform(output_features.detach().numpy())


OutOfMemoryError: CUDA out of memory. Tried to allocate 10.56 GiB (GPU 0; 8.00 GiB total capacity; 2.33 MiB already allocated; 6.14 GiB free; 18.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
import matplotlib.pyplot as plt
plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=labels)
plt.show()


NameError: name 'reduced_features' is not defined